### Importing dependencies


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

### Device config


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


print("Using device:", device)

### Data Collection and Analysis


In [ ]:
breast_cancer_dataset = sklearn.datasets.load_breast_cancer()

In [ ]:
print(breast_cancer_dataset)
print("keys: ", breast_cancer_dataset.keys())

In [ ]:
data_frame = pd.DataFrame(
    breast_cancer_dataset['data'], columns=breast_cancer_dataset['feature_names'])

In [ ]:
data_frame.head()

In [ ]:
data_frame['target'] = breast_cancer_dataset['target']

In [ ]:
data_frame.shape
data_frame.info()

In [ ]:
data_frame.describe()

In [ ]:
data_frame['target'].value_counts()

1 -> Benign \
0 -> Malignant

### Splitting the data


In [ ]:
X = data_frame.drop(labels='target', axis=1)
y = data_frame['target'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

### Standardize the data


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#### Distribution of the first feature and mean


In [ ]:
feature_number = 0

In [ ]:
# Plot a histogram of the first feature
plt.hist(X_train[breast_cancer_dataset.feature_names[0]],
         bins=20, color='blue', alpha=0.7)
plt.title("Distribution of " + breast_cancer_dataset.feature_names[0])
plt.xlabel(breast_cancer_dataset.feature_names[0])
plt.ylabel("Frequency")
plt.show()

In [ ]:
print("mean of the first feature before scaling",
      X_train[breast_cancer_dataset.feature_names[0]].mean())

In [ ]:
# Plot a histogram of the first feature
data = []

for i in range(0, len(X_train_scaled)):
    data.append(X_train_scaled[i][feature_number])

plt.hist(data, bins=20, color='blue', alpha=0.7)
plt.title("Distribution of " + breast_cancer_dataset.feature_names[0])
plt.xlabel(breast_cancer_dataset.feature_names[0])
plt.ylabel("Frequency")
plt.show()

In [ ]:
print("mean of the first feature after scaling",
      X_train_scaled[feature_number].mean())

### Converting the data to tensors


In [ ]:
X_train_tensor = torch.tensor(
    X_train_scaled, dtype=torch.float32).to(device=device)
X_test_tensor = torch.tensor(
    X_test_scaled, dtype=torch.float32).to(device=device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device=device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device=device)

### Neural Network architecture


In [ ]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

#### Hyperparameters


In [ ]:
input_size = X_train.shape[1]
hidden_size = 30
output_size = 1
learning_rate = 0.001
num_of_epochs = 100

In [ ]:
print(input_size)
print(X_train_tensor.shape[1])

#### Initialize the model


In [ ]:
model = NeuralNet(input_size, hidden_size, output_size).to(device)

#### Loss and optimizer


In [ ]:
criterion = nn.BCELoss()  # BCE because of binary classification
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

#### Training


In [ ]:
for epoch in range(num_of_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model.forward(X_train_tensor)

    # applying view method to convert from 1D to 2D with 1 column and retaining the same num of rows using -1
    loss = criterion(outputs, y_train_tensor.view(-1, 1))
    loss.backward()
    optimizer.step()

    # Calculate accuracy
    with torch.no_grad():
        prediction = outputs.round()  # to convert to either 0 or 1
        correct_prediction = (
            prediction == y_train_tensor.view(-1, 1)).float().sum()
        accuracy = correct_prediction / y_train_tensor.size(0)

    if (epoch+1) % 10 == 0:
        print(
            f"Epoch number: {epoch + 1}/{num_of_epochs}, Loss: {loss.item():.4f}, Accuracy: {accuracy.item() * 100:.2f}%")

### Model Evaluation

In [ ]:
model.eval()
with torch.no_grad():
    outputs = model(X_train_tensor)
    prediction = outputs.round()

    correct_prediction = (
        prediction == y_train_tensor.view(-1, 1)).float().sum()
    accuracy = correct_prediction / y_train_tensor.size(0)
    print(f"Accuracy on training data: {accuracy.item() * 100:.2f}%")

    # True positives, false positives, false negatives for malignant (0)
    TP = ((prediction == 0) & (y_train_tensor.view(-1, 1) == 0)).float().sum()
    FP = ((prediction == 0) & (y_train_tensor.view(-1, 1) == 1)).float().sum()
    FN = ((prediction == 1) & (y_train_tensor.view(-1, 1) == 0)).float().sum()

    # Precision, Recall, and F1 Score for malignant class
    precision = TP / (TP + FP + 1e-8)
    recall = TP / (TP + FN + 1e-8)
    f1_score = 2 * (precision * recall) / (precision + recall + 1e-8)

    print(f"Precision on training data: {precision * 100:.4f}%")
    print(f"Recall on training data: {recall * 100:.4f}%")
    print(f"F1 Score on training data: {f1_score.item() * 100:.4f}%")


In [ ]:
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    prediction = outputs.round()

    correct_prediction = (
        prediction == y_test_tensor.view(-1, 1)).float().sum()
    accuracy = correct_prediction / y_test_tensor.size(0)
    print(f"Accuracy on test data: {accuracy.item() * 100:.2f}%")

    # True positives, false positives, false negatives for malignant (0)
    TP = ((prediction == 0) & (y_test_tensor.view(-1, 1) == 0)).float().sum()
    FP = ((prediction == 0) & (y_test_tensor.view(-1, 1) == 1)).float().sum()
    FN = ((prediction == 1) & (y_test_tensor.view(-1, 1) == 0)).float().sum()

    # Precision, Recall, and F1 Score for malignant class
    precision = TP / (TP + FP + 1e-8)
    recall = TP / (TP + FN + 1e-8)
    f1_score = 2 * (precision * recall) / (precision + recall + 1e-8)

    print(f"Precision on training data: {precision * 100:.4f}%")
    print(f"Recall on training data: {recall * 100:.4f}%")
    print(f"F1 Score on training data: {f1_score.item() * 100:.4f}%")